https://medium.com/biaslyai/pytorch-introduction-to-neural-network-feedforward-neural-network-model-e7231cff47cb

In [1]:
%%capture
import torch
import copy
import matplotlib.pyplot as plt
import numpy as np
import os 
# import torchvision.models as models
# from torchvision import transforms
# from PIL import Image

In [2]:
class Feedforward(torch.nn.Module):
        def __init__(self, input_size, hidden_size):
            super(Feedforward, self).__init__()
            self.input_size = input_size
            self.hidden_size  = hidden_size
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.relu = torch.nn.ReLU()
            self.fc2 = torch.nn.Linear(self.hidden_size, 1)
            self.sigmoid = torch.nn.Sigmoid()
        def forward(self, x):
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
            output = self.sigmoid(output)
            return output

In [18]:
model = Feedforward(2, 10)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

In [4]:
# get previous last layer name
named_layers = dict(model.named_modules())
layers = list(named_layers.keys())

# too many branches, so just get the converged branch points
# '' is first layer, the input, so disregard it
layers = [x for x in layers if '.' not in x and x != '']  

In [5]:
layers

['fc1', 'relu', 'fc2', 'sigmoid']

In [7]:
# CREATE RANDOM DATA POINTS
from sklearn.datasets import make_blobs
def blob_label(y, label, loc): # assign labels
    target = np.copy(y)
    for l in loc:
        target[y == l] = label
    return target
x_train, y_train = make_blobs(n_samples=40, n_features=2, cluster_std=1.5, shuffle=True)
x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(blob_label(y_train, 0, [0]))
y_train = torch.FloatTensor(blob_label(y_train, 1, [1,2,3]))
x_test, y_test = make_blobs(n_samples=10, n_features=2, cluster_std=1.5, shuffle=True)
x_test = torch.FloatTensor(x_test)
y_test = torch.FloatTensor(blob_label(y_test, 0, [0]))
y_test = torch.FloatTensor(blob_label(y_test, 1, [1,2,3]))

In [19]:
model.eval()
y_pred = model(x_test)
before_train = criterion(y_pred.squeeze(), y_test)
print('Test loss before training' , before_train.item())

Test loss before training 0.6539065837860107


In [20]:
model.train()
epoch = 20

for epoch in range(epoch):
    optimizer.zero_grad()
    # Forward pass
    y_pred = model(x_train)
    # Compute Loss
    loss = criterion(y_pred.squeeze(), y_train)
   
    print('Epoch {}: train loss: {}'.format(epoch, loss.item()))
    # Backward pass
    loss.backward()
    optimizer.step()

Epoch 0: train loss: 0.9788061380386353
Epoch 1: train loss: 0.9468582272529602
Epoch 2: train loss: 0.9171887636184692
Epoch 3: train loss: 0.8895515203475952
Epoch 4: train loss: 0.863621711730957
Epoch 5: train loss: 0.8392621874809265
Epoch 6: train loss: 0.8165897130966187
Epoch 7: train loss: 0.7953654527664185
Epoch 8: train loss: 0.7753307819366455
Epoch 9: train loss: 0.7564679980278015
Epoch 10: train loss: 0.7386595010757446
Epoch 11: train loss: 0.7218672633171082
Epoch 12: train loss: 0.7059515714645386
Epoch 13: train loss: 0.6907398104667664
Epoch 14: train loss: 0.6762267351150513
Epoch 15: train loss: 0.6624606251716614
Epoch 16: train loss: 0.6494895219802856
Epoch 17: train loss: 0.6371070146560669
Epoch 18: train loss: 0.6252530217170715
Epoch 19: train loss: 0.6138989925384521


In [21]:
model.eval()
y_pred = model(x_test)
after_train = criterion(y_pred.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

Test loss after Training 0.6083815693855286




---
# Get Activations


In [9]:
x_train[0]

tensor([ 8.0379, -2.2177])

In [23]:
y_train[0]

tensor(0.)

In [22]:
# input = torch.randn(1, 1, 32, 32)
input = x_train[0]
out = model(input)
print(out)

tensor([0.5239], grad_fn=<SigmoidBackward0>)


In [10]:
def get_activations(input, layer_name):
    activation = {}
    def get_activation(name):
        def hook(model, input, output):
            activation[name] = output.detach()
        return hook

    for name_to_check, layer in model.named_modules():
        if name_to_check == layer_name:
            break
    layer.register_forward_hook(get_activation(layer_name))
    
    output = model(input)

    return activation.copy()  #else will return the same actvs of model

In [30]:
for layer_name in layers:
    print(get_activations(input, layer_name))

{'fc1': tensor([-0.5620, -0.0619,  3.3061, -0.9872, -6.5899, -2.0132, -3.8343, -1.2347,
         5.9933,  3.7645])}
{'relu': tensor([0.0000, 0.0000, 3.3061, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 5.9933,
        3.7645])}
{'fc2': tensor([0.0956])}
{'sigmoid': tensor([0.5239])}




---
Now slightly modify the input and see what happens to each layer!

Try different modification levels


In [28]:
input_2 = x_train[0] + torch.tensor([0.1,0])
input_2

tensor([ 8.1379, -2.2177])

In [29]:
for layer_name in layers:
    print(get_activations(input_2, layer_name))

{'fc1': tensor([-0.5677, -0.0643,  3.3324, -0.9959, -6.6545, -2.0499, -3.8789, -1.2428,
         6.0599,  3.8277])}
{'relu': tensor([0.0000, 0.0000, 3.3324, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 6.0599,
        3.8277])}
{'fc2': tensor([0.0961])}
{'sigmoid': tensor([0.5240])}




---



In [31]:
input_3 = x_train[0] + torch.tensor([1,0])
for layer_name in layers:
    print(get_activations(input_3, layer_name))

{'fc1': tensor([-0.6189, -0.0856,  3.5693, -1.0742, -7.2359, -2.3800, -4.2807, -1.3152,
         6.6597,  4.3965])}
{'relu': tensor([0.0000, 0.0000, 3.5693, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 6.6597,
        4.3965])}
{'fc2': tensor([0.1010])}
{'sigmoid': tensor([0.5252])}


Find a local motif. This is a local circuit?